# Creating a Random Forest Classification Model. 

## Bottom Line Up Front: 

The correlation of the features are too low to get more accurate predictions. Currently predicting at 54% at best, might as well be a coin flip to predict who would churn or not.

 We need to collect better data. 

***Recommendation***

I suggest the following: 
    - Actual Income amount, not just Low/Medium/High. This would have helped narrow down the tax bracket the customers who churn fall in, and tease out a potential reason for churning. 
    - Monthly spending to see if it is increasing, decreasing or flat.   Might offer insight into predicting who will churn. 
    - Interaction Quality not just count or resolved. Would answer how the customer felt when it was resolved. Ties in with Monthly Spending, did their spending decrease after an issue was reported, or after it stayed unresolved. 
    - Check if feedback is positive or negative. 
    - Consider noSQL to store what the feedback or complaint is, what are they inquiring about to see if customers who churn have the same issue.
    - When was the issue reported, when was it resolved. Time to resolution would help me understand better if our customer wait times might be a problem. 
    - Login dates, not just frequency of login, would be helpful to see if they were logging in a lot at first then suddenly stopped, and maybe we can find out why they stopped by checking the correlation with the above feature recommendations. 

The current dataset captures what customers do but not how they feel about the service. I can calculate Customer Satisfaction Scores from the above features and see how that correlates to churn. 

With the actual income, I can determine what customers earning churn because from the clustering data, it seems like the lower earners, who spend the most are the ones leaving more so that those who login frequently and spend and complain less. See previous report for more detail. 

## Algorithm Selection
Using Random Forest because while accuracy is important, the task specifies that the model must be interpretable to the stakeholders.
Random Forest indicates which features mattered most in the decisions and we can show this to stakeholders that this is why the model made a prediction.

XG Boost would be the most accurate, but lacks the interpretable aspect, with each decision tree correcting the one before it, tracing the specific prediction becomes tangled. Logistic Regression will be the easiest to explain but will not be as accurate as random forest. 

## Implementation:
    - I will train the model on 80% of the available Customer_Data_Cleaned set and test it on the remaining 20%.

    - target (y) is ChurnStatus. 

    - features (x) are all features except CustomerID because it's not important to the analysis, has no bearing whatsoever. 



In [1]:
# install libraries
!python -m pip install --upgrade pip -q
!pip install pandas numpy matplotlib seaborn scikit-learn openpyxl -q
!pip install --upgrade openpyxl -q
!pip install xgboost -q


In [2]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split    
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
pd.set_option('display.max_columns', None) # Set option to display all columns
pd.set_option('display.float_format', '{:.2f}'.format) # Set float format to 2 decimal places

# Load the cleaned customer data we previously created, and verify the info and the first few rows
Customer_Data = pd.read_excel('Customer_Data_Cleaned.xlsx')
Customer_Data.info()
Customer_Data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CustomerID                   1000 non-null   int64  
 1   Age                          1000 non-null   float64
 2   isMale                       1000 non-null   int64  
 3   IncomeLevel                  1000 non-null   int64  
 4   ChurnStatus                  1000 non-null   int64  
 5   TotalSpent                   1000 non-null   float64
 6   MinTransaction               1000 non-null   float64
 7   MaxTransaction               1000 non-null   float64
 8   TransactionFrequency         1000 non-null   float64
 9   LoyaltyLength                1000 non-null   float64
 10  InquiryCount                 1000 non-null   float64
 11  InquiryResolved              1000 non-null   float64
 12  InquiryUnresolved            1000 non-null   float64
 13  FeedbackCount      

,CustomerID,Age,isMale,IncomeLevel,ChurnStatus,TotalSpent,MinTransaction,MaxTransaction,TransactionFrequency,LoyaltyLength,InquiryCount,InquiryResolved,InquiryUnresolved,FeedbackCount,FeedbackResolved,FeedbackUnresolved,ComplaintCount,ComplaintResolved,ComplaintUnresolved,LoginFrequency,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Widowed,ServiceUsage_Mobile App,ServiceUsage_Online Banking,ServiceUsage_Website,TotalUnresolved,Cluster
0,1,1.23,1,1,0,-1.15,3.06,0.25,-1.56,-1.89,1.34,2.09,-0.39,-0.62,-0.47,-0.41,-0.61,-0.41,-0.44,0.58,0,0,1,0,1,0,0,-1.23,1
1,2,1.43,1,1,1,0.38,-0.52,0.07,0.75,0.96,1.34,2.09,-0.39,-0.62,-0.47,-0.41,-0.61,-0.41,-0.44,-1.49,0,1,0,0,0,0,1,-1.23,8
2,3,-1.66,1,1,0,0.59,-0.55,0.28,0.36,0.28,1.34,2.09,-0.39,-0.62,-0.47,-0.41,-0.61,-0.41,-0.44,-1.63,0,0,1,0,0,0,1,-1.23,8
3,4,-1.46,1,1,0,-0.47,-0.62,-0.07,-0.02,0.10,3.27,2.09,2.41,-0.62,-0.47,-0.41,-0.61,-0.41,-0.44,-1.70,0,0,0,1,0,0,1,1.57,8
4,5,-1.46,1,2,0,0.99,-0.37,0.80,1.13,0.86,-0.59,-0.42,-0.39,-0.62,-0.47,-0.41,-0.61,-0.41,-0.44,1.07,1,0,0,0,0,0,1,-1.23,0


In [4]:
# Separate features and target variable

X = Customer_Data.drop(columns=['CustomerID', 'ChurnStatus'])
y = Customer_Data['ChurnStatus']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, # 20% for testing
                                                    random_state=42, # makes it reproducible same split every time
                                                    stratify= y # keep churn ratio distributed and balanced in both sets
                                                    )

# # Verify the shape of the splits
# print("X_train shape:", X_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_train shape:", y_train.shape)
# print("y_test shape:", y_test.shape)

***Reasoning:***

When creating the for loop to check for hyperparameters, I could have used GridSearchCV but I wanted to evaluate for false alarms as well. 


In [ ]:
# Create the for loop to tune hyperparameters
best_recall = 0
best_settings = {}

for n_trees in [100, 200, 500]: # select number of trees
    for depth in [5, 10, 20, None]: # select max depth
        for weight in [1, 2, 5, 10]: # select class weight for churners
            for threshold in [0.3, 0.4, 0.5]: # select probability threshold
                model = RandomForestClassifier(n_estimators=n_trees, 
                                                max_depth=depth, 
                                                class_weight={0: 1, 1: weight},
                                                random_state=42) # random state for reproducibility
                model.fit(X_train, y_train)
                probs = model.predict_proba(X_test)[:, 1]
                predictions = (probs >= threshold).astype(int)
                
                caught = confusion_matrix(y_test, predictions)[1, 1]
                false_alarms = confusion_matrix(y_test, predictions)[0, 1]
                recall = caught / 41 # there are 41 actual churners in the test set, caught divided by total actual churners
                
                # Only save if this is the best so far AND false alarms are reasonable
                if recall > best_recall and false_alarms < 80: # stop if more than 80 false alarms, store settings
                    best_recall = recall
                    best_settings = {
                        'trees': n_trees,
                        'depth': depth,
                        'weight': weight,
                        'threshold': threshold,
                        'caught': caught,
                        'false_alarms': false_alarms
                    }

print("Best settings:", best_settings)
print(f"Recall: {best_recall:.2f}")

In [ ]:


# Build the RandomForestClassifer class on the best found parameters

model = RandomForestClassifier(n_estimators=500, max_depth = 10, class_weight={0:1, 1:5}, random_state=42).fit(X_train, y_train)
probability = model.predict_proba(X_test)[:, 1]
threshold = 0.30
predictions = (probability >= threshold).astype(int)
accuracy = accuracy_score(y_test, predictions)
class_report = classification_report(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
print(f'Accuracy: {accuracy:.2f}')
print("Classification Report:\n", class_report)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
# Check feature importance
importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)
display(importance)

***Observations:*** The features have no real weight. It tracks because of the low correlation across the board.
***Why it Matters:*** The data collection is insufficient, please see recommendations at the top of the report. 

***Additional Investigation***
After creating clusters, I add the Cluster to the Data to see if it would increase accuracy. It did but barely. No decrease in the false alarms.

***Reasoning: ***
Maybe the module can see a pattern between the clusters that would increase accuracy


In [ ]:
# Add cluster to features
X_with_cluster = X.copy()
X_with_cluster['Cluster'] = Customer_Data['Cluster']

# Split again with new features
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_with_cluster, y, test_size=0.2, random_state=42, stratify=y
)

# Train Random Forest with clusters
model = RandomForestClassifier(n_estimators=500, max_depth=10, 
                                class_weight={0:1, 1:5}, random_state=42)
model.fit(X_train_c, y_train_c)
probs = model.predict_proba(X_test_c)[:, 1]
predictions = (probs >= 0.3).astype(int)

print("WITH clusters:")
print(confusion_matrix(y_test_c, predictions))
print(f"Churners caught: {confusion_matrix(y_test_c, predictions)[1,1]}/41")

Because of how inaccurate the model using RandomForest is, I decided to check it against XGBoost. The results were similar. Only 2 more caught. 

In [ ]:
from xgboost import XGBClassifier

best_recall = 0
best_settings = {}

for n_trees in [100, 200, 500]:
    for depth in [3, 5, 10]:
        for weight in [5, 10, 15, 20]:
            for threshold in [0.2, 0.3, 0.4, 0.5]:
                model = XGBClassifier(n_estimators=n_trees, 
                                       max_depth=depth, 
                                       scale_pos_weight=weight,
                                       random_state=42)
                model.fit(X_train, y_train)
                probs = model.predict_proba(X_test)[:, 1]
                predictions = (probs >= threshold).astype(int)
                
                caught = confusion_matrix(y_test, predictions)[1, 1]
                false_alarms = confusion_matrix(y_test, predictions)[0, 1]
                recall = caught / 41
                
                if recall > best_recall and false_alarms < 80:
                    best_recall = recall
                    best_settings = {
                        'trees': n_trees,
                        'depth': depth,
                        'weight': weight,
                        'threshold': threshold,
                        'caught': caught,
                        'false_alarms': false_alarms
                    }

print("Best XGBoost settings:", best_settings)
print(f"Recall: {best_recall:.2f}")

In [ ]:
model = XGBClassifier(n_estimators=200, 
                       max_depth=3, 
                       scale_pos_weight=5,
                       random_state=42)
model.fit(X_train, y_train)
probs = model.predict_proba(X_test)[:, 1]
predictions = (probs >= 0.2).astype(int)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.2f}')
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

In [ ]:
model_xgb = XGBClassifier(n_estimators=200, max_depth=3, 
                           scale_pos_weight=5, random_state=42)
model_xgb.fit(X_train_c, y_train_c)
probs = model_xgb.predict_proba(X_test_c)[:, 1]
predictions = (probs >= 0.2).astype(int)

print("XGBoost WITH clusters:")
print(confusion_matrix(y_test_c, predictions))
print(f"Churners caught: {confusion_matrix(y_test_c, predictions)[1,1]}/41")